In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load your Excel file into df
file_path = "/Users/ashikurrahmanshihab/GitHub/Hybrid-Demand-Forecasting-and-Reorder-Point-Planning-Using-AROPHET-in-the-Pharmaceutical-Industry/Alton_20_Sales Data.xlsx"
df = pd.read_excel(file_path)

In [ ]:
# --- Step 1: Data Preparation ---
df_clean = df.copy()
if df_clean.iloc[0, 0] == "Month":
    df_clean = df_clean.iloc[1:].reset_index(drop=True)
df_clean.columns = ['ds', 'y']
df_clean['y'] = df_clean['y'].astype(str).str.replace(r'[^0-9]', '', regex=True).replace('', np.nan).astype(float)
df_clean['ds'] = pd.to_datetime(df_clean['ds'])
df_clean = df_clean.sort_values('ds').reset_index(drop=True)

In [ ]:
# --- Step 2: Forecasting with Prophet ---
model = Prophet(yearly_seasonality=True)
model.fit(df_clean)
future = model.make_future_dataframe(periods=12, freq='MS')
forecast = model.predict(future)
forecast_2025 = forecast[forecast['ds'].dt.year == 2025].copy()

In [ ]:
# --- Step 3: Bootstrapped Confidence Intervals ---
df_with_yhat = pd.merge(df_clean, forecast[['ds', 'yhat']], on='ds')
df_with_yhat['residual'] = df_with_yhat['y'] - df_with_yhat['yhat']
residuals = df_with_yhat['residual'].values

boot_samples = []
for _ in range(1000):
    sampled_residuals = np.random.choice(residuals, size=12, replace=True)
    simulated = forecast_2025['yhat'].values + sampled_residuals
    boot_samples.append(simulated)
boot_array = np.array(boot_samples)
forecast_2025['boot_lower'] = np.percentile(boot_array, 2.5, axis=0)
forecast_2025['boot_upper'] = np.percentile(boot_array, 97.5, axis=0)
forecast_2025['boot_median'] = np.percentile(boot_array, 50, axis=0)

In [ ]:
# --- Step 4: Dynamic ROP Calculation (Prophet) ---
k = 1.0  # risk aversion parameter
forecast_2025['prophet_rop'] = forecast_2025['boot_median'] + k * (forecast_2025['boot_upper'] - forecast_2025['boot_median'])

In [ ]:
# --- Step 5: Forecast Error Calculation (Residual Analysis) ---
# Already done above as 'residuals'

In [ ]:
# --- Step 6: Residual Modeling with ARIMA ---
arima_model = ARIMA(df_with_yhat['residual'], order=(1,0,0)).fit()
arima_pred = arima_model.forecast(steps=12)
forecast_2025['arima_correction'] = arima_pred.values

In [ ]:
# --- Step 7: Hybrid Forecast Using AROPHET ---
forecast_2025['arophet'] = forecast_2025['yhat'] + forecast_2025['arima_correction']

In [ ]:
# --- Step 8: Final Dynamic ROP Using AROPHET ---
boot_arophet_samples = []
for _ in range(1000):
    boot_resid = np.random.choice(residuals, size=12, replace=True)
    sim = forecast_2025['arophet'].values + boot_resid
    boot_arophet_samples.append(sim)
boot_arophet_array = np.array(boot_arophet_samples)
forecast_2025['arophet_lower'] = np.percentile(boot_arophet_array, 2.5, axis=0)
forecast_2025['arophet_upper'] = np.percentile(boot_arophet_array, 97.5, axis=0)
forecast_2025['arophet_median'] = np.percentile(boot_arophet_array, 50, axis=0)
forecast_2025['arophet_rop'] = forecast_2025['arophet_median'] + k * (forecast_2025['arophet_upper'] - forecast_2025['arophet_median'])

In [ ]:
# --- Step 9: Forecast Accuracy Comparison ---
# Evaluate on available historical data (2023-2024)
actual = df_clean['y'].values
prophet_pred = forecast.loc[forecast['ds'].isin(df_clean['ds']), 'yhat'].values
arima_correction_hist = arima_model.predict(start=0, end=len(df_clean)-1)
arophet_pred = prophet_pred + arima_correction_hist

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

metrics = {
    'Model': ['Prophet', 'AROPHET'],
    'MAE': [
        mean_absolute_error(actual, prophet_pred),
        mean_absolute_error(actual, arophet_pred)
    ],
    'RMSE': [
        np.sqrt(mean_squared_error(actual, prophet_pred)),
        np.sqrt(mean_squared_error(actual, arophet_pred))
    ],
    'MAPE': [
        mape(actual, prophet_pred),
        mape(actual, arophet_pred)
    ],
    'R2': [
        r2_score(actual, prophet_pred),
        r2_score(actual, arophet_pred)
    ]
}
accuracy_df = pd.DataFrame(metrics)

In [ ]:
# --- Prophet Forecast Plot with Error Bars ---
plt.figure(figsize=(12, 5))
plt.plot(df_clean['ds'], df_clean['y'], label='Actual', marker='o')
plt.plot(forecast['ds'], forecast['yhat'], label='Prophet Forecast', linestyle='--')
plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='blue', alpha=0.2, label='Prophet CI')
plt.errorbar(forecast['ds'], forecast['yhat'], 
             yerr=[forecast['yhat']-forecast['yhat_lower'], forecast['yhat_upper']-forecast['yhat']], 
             fmt='none', ecolor='blue', alpha=0.3)
plt.title('Prophet Forecast vs Actual (with Error Bars)')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# --- ARIMA Residual Correction Plot ---
plt.figure(figsize=(12, 5))
plt.plot(df_with_yhat['ds'], df_with_yhat['residual'], label='Prophet Residuals', marker='o')
plt.plot(pd.date_range(df_with_yhat['ds'].iloc[-1], periods=13, freq='MS')[1:], arima_pred, label='ARIMA Correction (2025)', linestyle='--')
plt.title('ARIMA Residual Correction')
plt.xlabel('Date')
plt.ylabel('Residual / Correction')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# --- AROPHET Hybrid Forecast Plot with Error Bars ---
plt.figure(figsize=(12, 5))
plt.plot(df_clean['ds'], df_clean['y'], label='Actual', marker='o')
plt.plot(forecast_2025['ds'], forecast_2025['arophet_median'], label='AROPHET Forecast', linestyle='-')
plt.fill_between(forecast_2025['ds'], forecast_2025['arophet_lower'], forecast_2025['arophet_upper'], color='gray', alpha=0.2, label='AROPHET CI')
plt.errorbar(forecast_2025['ds'], forecast_2025['arophet_median'], 
             yerr=[forecast_2025['arophet_median']-forecast_2025['arophet_lower'], forecast_2025['arophet_upper']-forecast_2025['arophet_median']], 
             fmt='none', ecolor='gray', alpha=0.3)
plt.title('AROPHET Hybrid Forecast (with Error Bars)')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# --- ROP Comparison Plot ---
plt.figure(figsize=(12, 5))
plt.plot(forecast_2025['ds'], forecast_2025['prophet_rop'], label='Prophet ROP', linestyle='--')
plt.plot(forecast_2025['ds'], forecast_2025['arophet_rop'], label='AROPHET ROP', linestyle='-')
plt.title('ROP Comparison: Prophet vs AROPHET')
plt.xlabel('Date')
plt.ylabel('ROP')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# --- Forecast Errors Plot ---
prophet_errors = actual - prophet_pred
arophet_errors = actual - arophet_pred
plt.figure(figsize=(12, 5))
plt.plot(df_clean['ds'], prophet_errors, label='Prophet Errors', marker='o')
plt.plot(df_clean['ds'], arophet_errors, label='AROPHET Errors', marker='o')
plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.title('Forecast Errors: Actual - Predicted')
plt.xlabel('Date')
plt.ylabel('Error')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# --- Rolling Metrics Plot (e.g., Rolling MAE) ---
window = 6
rolling_mae_prophet = pd.Series(np.abs(prophet_errors)).rolling(window).mean()
rolling_mae_arophet = pd.Series(np.abs(arophet_errors)).rolling(window).mean()
plt.figure(figsize=(12, 5))
plt.plot(df_clean['ds'], rolling_mae_prophet, label=f'Prophet Rolling MAE ({window} months)')
plt.plot(df_clean['ds'], rolling_mae_arophet, label=f'AROPHET Rolling MAE ({window} months)')
plt.title('Rolling MAE Comparison')
plt.xlabel('Date')
plt.ylabel('Rolling MAE')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# --- (Optional) Interactive Plotly Plot ---
# import plotly.express as px
# fig = px.line(df_clean, x='ds', y='y', title='Actual Sales (Interactive)')
# fig.add_scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Prophet Forecast')
# fig.show()

In [ ]:
# --- Step 10: Visualization and Interpretation ---
plt.figure(figsize=(14, 6))
plt.plot(df_clean['ds'], df_clean['y'], label='Actual', marker='o')
plt.plot(forecast_2025['ds'], forecast_2025['boot_median'], label='Prophet Forecast', linestyle='--')
plt.plot(forecast_2025['ds'], forecast_2025['arophet_median'], label='AROPHET Forecast', linestyle='-')
plt.plot(forecast_2025['ds'], forecast_2025['prophet_rop'], label='Prophet ROP', linestyle='--')
plt.plot(forecast_2025['ds'], forecast_2025['arophet_rop'], label='AROPHET ROP', linestyle='-')
plt.fill_between(forecast_2025['ds'], forecast_2025['arophet_lower'], forecast_2025['arophet_upper'], color='gray', alpha=0.2, label='AROPHET CI')
plt.title("Forecast and ROP Comparison (Prophet vs AROPHET)")
plt.xlabel("Month")
plt.ylabel("Sales Quantity / ROP")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Residual diagnostics
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plot_acf(df_with_yhat['residual'], lags=12, ax=plt.gca())
plt.title('Residual ACF')
plt.subplot(1,2,2)
plot_pacf(df_with_yhat['residual'], lags=12, ax=plt.gca())
plt.title('Residual PACF')
plt.tight_layout()
plt.show()

lb_test = acorr_ljungbox(df_with_yhat['residual'], lags=[12], return_df=True)
print("Ljung-Box test p-value (lag 12):", lb_test['lb_pvalue'].values[0])

In [ ]:
# Show accuracy table and result table
print("Forecast Accuracy Comparison:")
print(accuracy_df)
print("\n2025 Forecast Table:")
print(forecast_2025[['ds', 'boot_median', 'prophet_rop', 'arophet_median', 'arophet_rop']])